<a href="https://colab.research.google.com/github/ShahzaibSE/langgraph-e2e-solutions/blob/main/2_1_tools_messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LLM Actions and Structure**

## Review

We built a simple graph with nodes, normal edges, and conditional edges.

## Goals

Now, let's understand the power of Actions with AI Models and before that structure our prompts.

## Messages

Chat models can use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages), which capture different roles within a conversation.

LangChain supports various message types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`.

These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call.

Let's create a list of messages.

Each message can be supplied with a few things:

* `content` - content of the message
* `name` - optionally, a message author
* `response_metadata` - optionally, a dict of metadata (e.g., often populated by model provider for `AIMessages`)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from pprint import pprint

messages = [
    AIMessage(content='Hi, I\'m your Virtual Assistant', name='Model')
]
#
messages.append(HumanMessage(content='What is the meaning of life?', name='Shazy'))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for message in messages:
  message.pretty_print()

================================== Ai Message ==================================
Name: Model

Hi, I'm your Virtual Assistant
================================ Human Message =================================
Name: Shazy

What is the meaning of life?
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about.
================================ Human Message =================================
Name: Lance

I want to learn about the best place to see Orcas in the US.


Let's use HumanMessage prompt for our model.

In [ ]:
from google.colab import userdata

In [ ]:
%pip install --quiet -U langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:

%pip install -U langchain
%pip install -U langchain-google-genai
%pip install -U langchain-community

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatO

In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Initialize the OpenAI LLM
openai_llm = OpenAI(
    temperature=0.6,
    openai_api_key=OPENAI_API_KEY
)


In [ ]:
# Invoking OpenAI model.
openai_llm.invoke("What is the meaning of life?")

'\n\nThe meaning of life is a philosophical question that has been debated by people for centuries. Some believe that the meaning of life is to seek happiness and fulfillment, while others believe it is to fulfill a certain purpose or destiny. Some see it as a spiritual journey, while others see it as a pursuit of knowledge and understanding. Ultimately, the meaning of life is subjective and can be different for each individual. It is up to each person to determine their own meaning and purpose in life.'